### Demo: Parially automated PPV database.

#### Human-generated and machine-generated data upload to Dataverse.


#### V3: Add a modified version of Yoni's code to create a form.
RLH 041824

#### V4: Add Dataverse integration from Felix.
RLH 042724

####  V5: Add a dropdown list to the form and populate it with Sample_ID's from the Dataverse table. Also added the code to call the 'insert-data.ps1' Powershell script and used it to insert data into the 'J-V_test_data_V2' table. Added support for optional image upload.   Added a lookup column with logical name 'cr69a_sampledatav2lookup'.  My efforts to insert the lookup from Powershell failed -- defering that to a later date.
RLH 043024

#### V6: Move the classes SampleDataverseTable and JVScanDataverseTable to a file called `db_dataverse.py'.
RLHY 051224

#### This is a screenshot the form for J-V test metadata:

####

<div>
<img src="attachment:form_screenshot_V2.png" width="250"/>
</div>

### 1. This is the sample data input form:

<div>
<img src="attachment:71caadf0-1c68-46d2-bc49-2f56f2507f63.png" width="350"/>
</div>

##### QR code for the sample data import:



<div>
<img src="attachment:88403105-bb18-4e27-af9a-db6ec4f3dbb2.png" width="250"/>
</div>



### 2. Felix's code to retrieve Sample ID's. 

This version returns a list of Sample_ID's from the table "Sample Data V2" (cr69a_sampledatav2) It ss in python, calling a Powershell script.  

#### `SampleDataverseTable` class.   050524 RLH. 

In [2]:
# Example usage:
from db_dataverse import SampleDataverseTable # Dataverse code.
table = SampleDataverseTable("cr69a_sampledatav2s")
sample_ids, recent_values, result = table.recent_entries(3)
print(sample_ids)
recent_values

['Hamamatsu_S1787-12', 'NiOx-11', '050124_test_randy_4']


,Sample ID,Operator,Perovskite Composition,HTL Material,ETL Material,Top Capping Material,Bottom Capping Material,Bulk Passivation Materials,Is Encapsulated
0,Hamamatsu_S1787-12,Randy,Silicon,,,,,,True
1,NiOx-11,Seid,FAPI,NiOx,C60/BCP,3MTPAI,PEAI,"[""MDACl2"",""MACl""]",True
2,050124_test_randy_4,Randy,FAPI,PTAA,SnO2,PEACl,None,"[""MDACl2""]",True


### 3. Yoni's J-V test metadata form: This version includes the dropdown.

RLH 043024

In [3]:
pwd

'c:\\Users\\RandallHeadrick\\OneDrive - University of Vermont\\Software_development\\K2401SSDB'

In [4]:
# Example usage:
import db_form  # Yoni's form code.
import ipywidgets as widgets
import json
import os

# The questions json file can be anywhere, but it will usually be in the directory with this Notebook.
questions_file_main= '../K2401SSDB/main_questions.json'
dir_path_local = '../K2401SSDB'
#dir_path_local = '../catcoders_2024/nonexistant'
#dir_path_local = '../../Jim_043024'
form = db_form.parameters_form(persistent=False, dir_path_local=dir_path_local, 
                               questions_file_main=questions_file_main, drop_down_values=[sample_ids])
form.dropdown_values = sample_ids

In [5]:
print(form.dropdown_values)

['Hamamatsu_S1787-12', 'NiOx-11', '050124_test_randy_4']


In [6]:
form.display_question_canvas()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Button(button_style='success', description='Submit', style=ButtonStyle())

Form responses: "['K2401ssdb', 'Hamamatsu_S1787-12', [False, True, False], 'Downstairs (W125)', '2', 'No', 'Yes', '55.0', '0.815']"


In [7]:
print(form.responses)

['K2401ssdb', 'Hamamatsu_S1787-12', [False, True, False], 'Downstairs (W125)', '2', 'No', 'Yes', '55.0', '0.815']


### 4. Collect the J-V data.

Here we will load old data that has been saved to a file as a demonstration.
The data was harvested from the data files on the laptop computer we use in the W426 lab.

#### Before we collect the J-V data we need to confirm that the sample chosen actually exists.

In [8]:
sample_id = form.responses[1]
#sample_id = "junkeddy_junk"
sample_id = "Test 11"
Sample_primary_ID = ''
allow_entry =  False
print(sample_id)

Test 11


In [12]:
# See the class DataverseTable for the function code.
table = SampleDataverseTable("cr69a_sampledatav2s")
allow_entry, Sample_primary_ID, check_existence_result = table.check_existence(sample_id)
# print(allow_entry)


 PowerShell: Invalid: Sample ID was not found. 

Sample ID 'Test 11' was not found in the table 'cr69a_sampledatav2s', please enter a new sample ID.

 PowerShell: Invalid: Sample ID was not found. 

Sample ID 'allow_entry, Sample_primary_ID, check_existence_result' was not found in the table 'cr69a_sampledatav2s', please enter a new sample ID.

 PowerShell: Found the Sample ID: 'Hamamatsu_S1787-12', 'Sample_data_V2_ID': f9cdbf0e-6f12-ef11-9f8a-000d3a55d192, 'Operator': Randy, 'Entry Date': 05/15/2024 03:50:00, 'Cell_active_area': 0.0672 


Confirmed the Sample ID: 'Hamamatsu_S1787-12'.


In [13]:
Sample_primary_ID

'f9cdbf0e-6f12-ef11-9f8a-000d3a55d192'

In [14]:
#  get the test data
import pandas as pd
cell_data_dataframe  = pd.read_csv('test_JV_data.csv')

In [15]:
cell_data_dataframe.head()

,Sample_ID,Elapsed Time,Test_ID,I_ph(suns),V_oc(V),mpp(V),J_sc(mA),R_sh,R_ser,FF,PCE(%)
0,NiOx_C2_77d_4F,5.597869,NiOx_C2_77d_4F_094305,0.805,1.074103,0.775,21.923187,266.104605,10.039115,0.464750,10.943837
1,NiOx_C2_77d_4R,5.597905,NiOx_C2_77d_4R_094305,0.805,1.080675,0.750,21.689500,582.101632,7.969562,0.513278,12.030875
2,NiOx_C2_77d_5F,7.150103,NiOx_C2_77d_5F_094439,0.805,1.081829,0.800,21.938013,294.705449,6.328962,0.482623,11.454176
3,NiOx_C2_77d_5R,7.150362,NiOx_C2_77d_5R_094439,0.805,1.101452,0.875,21.778660,853.001511,6.043041,0.592451,14.211803
4,NiOx_C2_77d_6F,8.706784,NiOx_C2_77d_6F_094612,0.805,1.076664,0.800,22.521313,423.929593,5.807517,0.500236,12.129664


### 5. Save to the J-V_test_data V2 table

* Starting point is form-to-dataverse.py by Felix
* A modification of 'insert_data.ps1' accepts 18 parameters, including from Yoni's form plus the machine generated J-V scan data.
* Upload the 17 parameters to the form to the table J-V_test_data  with plural name cr69a_jv_test_datas .
* Alternately we may use cr69a_jv_test_data_v2 or new_jv_tests
* new_jv_tests has a Lookup for the Sample name, which I don't know how to use yet.

042824  RLH


* Pardon the messy code, I'm going to join the user-supplied data with the machine-generated data.
* Then upload the record to the table `J-V_test_data`.
  
043024 RLH

####  Load the data (normally will be done with the solar simulator tester).

In [16]:
# Just the first row is enough for this test.
first_data_record = cell_data_dataframe[:1]
first_data_record.loc[0,'Sample_ID'] = form.responses[1]
first_data_record

,Sample_ID,Elapsed Time,Test_ID,I_ph(suns),V_oc(V),mpp(V),J_sc(mA),R_sh,R_ser,FF,PCE(%)
0,Hamamatsu_S1787-12,5.597869,NiOx_C2_77d_4F_094305,0.805,1.074103,0.775,21.923187,266.104605,10.039115,0.46475,10.943837


#### The form responses contain the 9  user-supplied data fields.

In [17]:
form.responses

['K2401ssdb',
 'Hamamatsu_S1787-12',
 [False, True, False],
 'Downstairs (W125)',
 '2',
 'No',
 'Yes',
 '55.0',
 '0.815']

In [18]:
print("User-supplied data:\n", form.responses)
[operator, sample_ID, scan_type, lab_loc, cellnum, is_module, is_masked, mask_area, I_ph] =  form.responses
# Convert scan type to 'F', 'R', 'S' for Fwd, Rev, and Stabilized scan types.
sc_type_new = ""
if scan_type[0]:
    sc_type_new += 'F'
if scan_type[1]:
    sc_type_new += 'R'
if scan_type[2]:
    sc_type_new += 'S'
scan_type = sc_type_new

User-supplied data:
 ['K2401ssdb', 'Hamamatsu_S1787-12', [False, True, False], 'Downstairs (W125)', '2', 'No', 'Yes', '55.0', '0.815']


In [19]:
data = { 'Operator':    [operator],
        'Scan_type':    [scan_type],
        'Lab_location': [lab_loc],
        'Cell_number':  [cellnum],
        'Module':       [is_module],
        'Masked':       [is_masked],
        'Mask_area':    [mask_area]
        }
form_dataframe = pd.DataFrame(data)
form_dataframe

,Operator,Scan_type,Lab_location,Cell_number,Module,Masked,Mask_area
0,K2401ssdb,R,Downstairs (W125),2,No,Yes,55.0


In [20]:
first_data_record

,Sample_ID,Elapsed Time,Test_ID,I_ph(suns),V_oc(V),mpp(V),J_sc(mA),R_sh,R_ser,FF,PCE(%)
0,Hamamatsu_S1787-12,5.597869,NiOx_C2_77d_4F_094305,0.805,1.074103,0.775,21.923187,266.104605,10.039115,0.46475,10.943837


#### The joining happens here.

In [21]:
# The joining happens here.
first_data_record = first_data_record.join(form_dataframe)

In [22]:
# Finally, I have all the data that I need, ready to upload to dataverse.
first_data_record.loc[0]

Sample_ID          Hamamatsu_S1787-12
Elapsed Time                 5.597869
Test_ID         NiOx_C2_77d_4F_094305
I_ph(suns)                      0.805
V_oc(V)                      1.074103
mpp(V)                          0.775
J_sc(mA)                    21.923187
R_sh                       266.104605
R_ser                       10.039115
FF                            0.46475
PCE(%)                      10.943837
Operator                    K2401ssdb
Scan_type                           R
Lab_location        Downstairs (W125)
Cell_number                         2
Module                             No
Masked                            Yes
Mask_area                        55.0
Name: 0, dtype: object

#### JVScanDataverseTable class.  In this version the data is added to the table with the insert_data() function. It calls the powershell script and passes the data values.

In [23]:
del  JVScanDataverseTable

In [24]:
# Example usage:
# Initialize the JVScanDataverseTable instance with environment-specific parameters.
# This is for the table J-V_test_data  with logical name cr69a_jv_test_data.
from db_dataverse import JVScanDataverseTable

# Example data row as a dictionary.
row = {
    'sample_id': 'Test 1',
    'elapsed_time': '5.60',
    'base_time': '0.10',  # New base_time value.
    'test_id': 'NiOx_C2_77d_4F_094305c',
    'i_ph_suns': '0.00',
    'voc_v': '1.07',
    'mpp_v': '0.00',
    'jsc_ma': '0.00',
    'rsh': '266.10',
    'rser': '10.04',
    'ff': '0.46',
    'pce': '10.94',
    'operator': 'Randy',
    'scan_type': 'S',
    'lab_location': 'Upstairs (W426)',
    'cell_number': '3',
    'module': 'Yes',
    'masked': 'No',
    'mask_area': '100.10',
    'temp_c': '21',  # New.
    'hum_pct': '10', # New.
    'four_wire_mode': 'No', # New.
    'scan_data_path': 'testx21' # New.
}

# Columnn logical names. 
col_logical_names = {
    'sample_id': 'cr69a_sample_id',
    'elapsed_time': 'cr69a_elapsed_time_min',
    'base_time': 'cr69a_base_time_sec',  # New base_time value.
    'test_id': 'cr69a_test_id',
    'i_ph_suns': 'cr69a_iph_suns',
    'voc_v': 'cr69a_voc_v',
    'mpp_v': 'cr69a_mpp_v',
    'jsc_ma': 'cr69a_jsc_macm2',
    'rsh': 'cr69a_rsh',
    'rser': 'cr69a_rser',
    'ff': 'cr69a_ff_',
    'pce': 'cr69a_pce_',
    'operator': 'cr69a_operator_name',
    'scan_type': 'cr69a_scan_type',
    'lab_location': 'cr69a_location',
    'cell_number': 'cr69a_cell_number',
    'module': 'cr69a_module',
    'masked': 'cr69a_masked',
    'mask_area': 'cr69a_mask_area_cm2',
    'temp_c': 'cr69a_temperature_c',  # New.
    'hum_pct': 'cr69a_humidity_pct', # New.
    'four_wire_mode': 'cr69a_wire_mode', # New.
    'scan_data_path': 'cr69a_path' # New.
}

# Check for consistency between the row data and the column logical names.
res = all(col_logical_names.get(key) != None for key in row)

if res != True:
    print("Error:  'row' and 'col_logical_names' keys don't match.")
else:
    # Create an instance of the JVScanDataverseTable with environment-specific parameter data.
    table = JVScanDataverseTable(
        crm_url='https://orgc25b23b3.api.crm.dynamics.com/',
        table_name='cr69a_jv_test_data_v2s',
        col_logical_names =  col_logical_names
    )
    # Insert data into the table.  Make sure the 'row' dictionary is consistent with the 'self.cols' parameter of JVScanDataverseTable.
    # To do:  we might one day also pass 'cols' as a dictionary with the same keys. Then the class constructor can make sure they are consistent.
    #table.insert_data(row)  # Simple version with only the required rows. 
    returnval = table.insert_data(row, image_path= os.getcwd() +'/93_5F_152003.png', image_column_name='cr69a_scan_image')
    # Inserting the Lookup always produces and error.  RLH 051224.
    #table.insert_data(row, sample_lookup_id='01e2d5d8-2007-ef11-9f89-000d3a55d192', sample_lookup_name='cr69a_sample_data_lookup')
    print(returnval)

Hello world!
CompletedProcess(args='pwsh -ExecutionPolicy Bypass -File "scripts/insert-data_v4.ps1" -crm_url "https://orgc25b23b3.api.crm.dynamics.com/" -table_name "cr69a_jv_test_data_v2s" -cols "cr69a_sample_id,cr69a_elapsed_time_min,cr69a_base_time_sec,cr69a_test_id,cr69a_iph_suns,cr69a_voc_v,cr69a_mpp_v,cr69a_jsc_macm2,cr69a_rsh,cr69a_rser,cr69a_ff_,cr69a_pce_,cr69a_operator_name,cr69a_scan_type,cr69a_location,cr69a_cell_number,cr69a_module,cr69a_masked,cr69a_mask_area_cm2,cr69a_temperature_c,cr69a_humidity_pct,cr69a_wire_mode,cr69a_path" -sample_id "Test 1" -elapsed_time "5.60" -base_time "0.10" -test_id "NiOx_C2_77d_4F_094305c" -i_ph_suns "0.00" -voc_v "1.07" -mpp_v "0.00" -jsc_ma "0.00" -rsh "266.10" -rser "10.04" -ff "0.46" -pce "10.94" -operator "Randy" -scan_type "S" -lab_location "Upstairs (W426)" -cell_number "3" -module "Yes" -masked "No" -mask_area "100.10" -temp_c "21" -hum_pct "10" -four_wire_mode "No" -scan_data_path "testx21" -image_path "c:\\Users\\RandallHeadrick\\O

In [23]:
ls

 Volume in drive C is Local Disk
 Volume Serial Number is 364F-B4C4

 Directory of C:\Users\RandallHeadrick\OneDrive - University of Vermont\Software_development\K2401SSDB

05/12/2024  11:47 PM    <DIR>          .
05/06/2024  02:45 PM    <DIR>          ..
05/06/2024  02:35 PM    <DIR>          .ipynb_checkpoints
05/12/2024  11:44 PM    <DIR>          __pycache__
05/06/2024  02:49 PM          (23,817) 93_5F_152003.pdf
03/02/2022  04:20 PM            16,881 93_5F_152003.png
05/12/2024  03:17 PM          (13,575) Checklists.xlsx
05/12/2024  11:41 PM             9,147 db_dataverse.py
05/12/2024  11:43 PM             7,336 db_form.py
05/06/2024  02:51 PM           (3,715) download.png
05/05/2024  12:20 AM       (1,473,318) Form_test_V5b.ipynb
05/12/2024  07:32 PM         1,505,956 Form_test_V5c.ipynb
05/12/2024  11:47 PM         1,481,929 Form_test_V6.ipynb
05/12/2024  11:44 PM             1,094 K2401SSDB_questions_latest.json
04/30/2024  01:06 PM             1,529 main_questions.json
05/08

##### End of test script!

In [29]:
import subprocess

cli = ('pwsh -ExecutionPolicy Bypass -File "scripts/insert-data_new.ps1" "Test 1" "0.1"')
subprocess.run(cli, shell=True)

CompletedProcess(args='pwsh -ExecutionPolicy Bypass -File "scripts/insert-data_new.ps1" "Test 1" "0.1"', returncode=0)